In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/generation_config.json
/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv


In [2]:
# Instaliamo la libreria transformers
!pip install -q -U git+https://github.com/huggingface/transformers

In [3]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

model_path="/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer=AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
    trust_remote_code = True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
print("Hello")

Hello


In [5]:
def generate_essay(prompt):
    messages = [{
        "role":"user",
        "content": prompt
    }]

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors = "pt").to('cuda')
    
    # Setting `pad_token_id` to `eos_token_id` for open-ended generation.
    generated_ids = model.generate(
        model_inputs,
        max_new_tokens = 7500,
        do_sample = True,
        pad_token_id = tokenizer.eos_token_id
        
    )

    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    text = decoded[0].split("[/INST]")[1]
    return text


In [6]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path

path = Path("/kaggle/input/llm-detect-ai-generated-text/")

train_prompts = pd.read_csv(f"{path}/train_prompts.csv")

In [7]:
train_prompts


,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [8]:
train_essays = pd.read_csv(f"{path}/train_essays.csv")
train_essays['text_length'] = train_essays['text'].str.len()

text_len_mean = int(train_essays.query("generated == 0")['text_length'].mean())
text_len_std = int(train_essays.query("generated == 0")['text_length'].std())

print(f"Mean length of train essays by human in our dataset: {text_len_mean}")
print(f"Mean standard deviation of train essays by human in our dataset: {text_len_std}")

Mean length of train essays by human in our dataset: 3172
Mean standard deviation of train essays by human in our dataset: 918


In [9]:
# Defining a configuration object that contains all the parameters at one place.
class CFG:
    guidance_pct = 0.25 # number of times the instructions are shared with the LLM.
    n_essays = 3 # Modify this to generate as many essays you are interested in for each prompt

synthetic_essay_lengths = np.random.normal(text_len_mean, text_len_std, CFG.n_essays * 2).astype(int)
# multiplying by 2 because we want this number for all prompts.

In [ ]:
# count the number of parts

part = 0

In [10]:
import random
import string
llm_essays = []
essay_counter = 0

for p in train_prompts['prompt_id'].unique():
    for k in tqdm(range(CFG.n_essays), desc=f"prompt_id: {p}"):
        prompt_name = train_prompts.loc[p, 'prompt_name']
        inst = ""
        if np.random.random() < CFG.guidance_pct:
            inst = train_prompts.loc[p, 'instructions']
            prompt_combined = f'''
                You are a student working on the following assignment.

                Create an essay based on the following topic in no more than {synthetic_essay_lengths[essay_counter]} words.

                Topic: ''' \
                + prompt_name \
                + ".\n\n" \
                + "Instructions: \n\n" \
                + inst
        else:
            prompt_combined = f'''
                You are a student working on the following assignment.

                Create an essay based on the following topic in no more than {synthetic_essay_lengths[essay_counter]} words.

                Topic: ''' \
                + prompt_name \
                + ".\n\n" \
        
        # generate the essay
        essay_output = generate_essay(prompt_combined)
        essay_counter += 1
        
        data_output = {
            'id': ''.join(random.choices(string.ascii_letters + string.digits, k=10)),
            'prompt_id': p,
            'text': essay_output,
            'generated': 1
        }
        llm_essays.append(data_output)
    
llm_essays = pd.DataFrame(llm_essays)
part += 1
llm_essays

prompt_id: 0:   0%|          | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
prompt_id: 1: 100%|██████████| 3/3 [02:28<00:00, 49.54s/it]


,prompt_id,prompt_name,text,generated
0,0,Car-free cities,Car-free cities are becoming increasingly pop...,1
1,0,Car-free cities,Car-Free Cities: A Path to Sustainable Urban ...,1
2,0,Car-free cities,Car-free cities: A vision for the future\n\nT...,1
3,1,Does the electoral college work?,The electoral college is a system used by the...,1
4,1,Does the electoral college work?,The electoral college is a group of people wh...,1
5,1,Does the electoral college work?,Introduction:\n\nThe electoral college is a g...,1


In [ ]:
# Export the generated essays to a csv file using part as index
llm_essays.to_csv(f"{path}/llm_essays_{part}.csv", index=False)

